In [1]:
!pip install selenium
!pip install selenium_stealth
!pip install selenium webdriver-manager


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 18.7 MB/s eta 0:00:00


In [2]:
# import libraries
from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import pandas as pd
import numpy as np
import re
import time
import random
import os

In [3]:
# Initialize Selenium WebDriver with options
def initialize_driver_options():
    # create a ChromeOptions object
    options = Options()

    #run in headless mode
    options.add_argument("--headless")

    # disable the AutomationControlled feature of Blink rendering engine
    options.add_argument('--disable-blink-features=AutomationControlled')

    # disable pop-up blocking
    options.add_argument('--disable-popup-blocking')

    # start the browser window in maximized mode
    options.add_argument('--start-maximized')

    # disable extensions
    options.add_argument('--disable-extensions')

    # disable sandbox mode
    options.add_argument('--no-sandbox')

    # disable shared memory usage
    options.add_argument('--disable-dev-shm-usage')


    return options

In [4]:
# user agents list
user_agent_list = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
                  ]

In [18]:
# initialise driver options
options = initialize_driver_options()

# request for a random and different user agent everytime
user_agent = random.choice(user_agent_list)

options.add_argument(f'user-agent={user_agent}')

#initialise the fortified scraper
driver = webdriver.Chrome(options=options)

driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": user_agent})

#enable stealth mode
stealth(driver,
    languages=["en-US", "en"],
    vendor="Google Inc.",
    platform="Win32",
    webgl_vendor="Intel Inc.",
    renderer="Intel Iris OpenGL Engine",
    fix_hairline=True,
      )


In [19]:
# get desired category links
def get_cat_links(site):
  driver.get(site)
  time.sleep(2)
  links_list = []
  #fetch the navigation links in the bbc igbo site
  for el in driver.find_elements(By.XPATH, '/html/body/div[1]/div/header/nav/div/div[1]/div/ul'):
    for link in el.find_elements(By.TAG_NAME, 'a'):
      links = link.get_attribute('href')
      links_list.append(links)

  return links_list[1:3]

links = get_cat_links("https://www.bbc.com/igbo")
links

['https://www.bbc.com/igbo/topics/cnq68k0x2vrt',
 'https://www.bbc.com/igbo/topics/c3l19z3qjmyt']

In [36]:
dic = {"text":[], "category": [], "source": [], "no_of_paragraphs": []}
# scrape the desired categories
driver.get(links[0])
time.sleep(2)
link_list = []

# get category name
category = driver.find_element(By.TAG_NAME, "h1").text

# get category articles links
for el in driver.find_elements(By.XPATH, '/html/body/div[1]/div/div/main/div/div[2]/ul'):
    for link in el.find_elements(By.TAG_NAME, 'a'):
      pages = link.get_attribute('href')
      link_list.append(pages)

#texts = []
#counts = []
for i in link_list[0:2]:
  driver.get(i)
  time.sleep(2)
  article = ""
  for el in driver.find_elements(By.XPATH, "/html/body/div[1]/div/div/div/div/div[1]"):
    count = 0
    for p in el.find_elements(By.TAG_NAME, "div"):
      count += 1
      article += p.text
      article += "\n"
  dic["text"].append(article)
  dic["category"].append(category)
  dic["source"].append(i)
  dic["no_of_paragraphs"].append(count)
print(dic)




{'text': ['Ụfọdụ ihe i kwesịrị ịma gbasara \'Chairman\' Christian Chukwu\nEBE FOTO SI,\nGETTY IMAGES\n12 Eprel 2025\nMgbe ikpeazụ e tinyere ya ozi ọhụrụ 16 Eprel 2025\nMgbe ikpeazụ e tinyere ya ozi ọhụrụ 16 Eprel 2025\nA mụrụ ya n\'ụbọchị 4 Jenụwarị 1951 n\'Uwani dị n\'okpuru ọchịchị Enugu South n\'Enugwu steeti.\nỌ gụrụ akwụkwọ praịmarị n\'ụlọakwụkwọ Christ Church Primary School, Uwani ma gụọ nke sekọndịrị na National Secondary School Nike, Enugwu.\nỌ gbara bọọlụ n\'otu egwu bọọlụ Rangers nke Enugwu n\'afọ 1972 ma mechaa soro na ndị nlekọta otu egwu bọọlụ ahụ n\'afọ 1982.\nN\'afọ 1977, o duuru otu egwu bọọlụ Flying Antelopes gaa mabute iko mba Afrịka n\'asọmpi African Cup Winners Cup anaghị agbazi ugbua.\nChristian Chukwu bụ kaptịn Naịjirịa mbụ mabutere iko n\'asọmpi African Nations Cup oge ha jichara ọkpụ goolu atọ asataghị otu merie otu egwu bọọlụ mba Ọljirịa n\'afọ 1980.\nN\'afọ 1985, ọ rụrụ ọrụ dịka osote onye nchịkọta otu egwu bọọlụ Naịjirịa ndị akarịbeghị afọ iri na isii (U-16) 

In [37]:
df = pd.DataFrame(dic)
df

,text,category,source,no_of_paragraphs
0,Ụfọdụ ihe i kwesịrị ịma gbasara 'Chairman' Chr...,Egwuruegwu,https://www.bbc.com/igbo/articles/c793q1dql32o,32
1,LIVE\nSTART\nLIVE\nVideo quality\nLow\nMedium\...,Egwuruegwu,https://www.bbc.com/igbo/articles/cdxq4yw4117o,24


In [31]:
texts = []
counts = []
for i in link_list[0:2]:
  driver.get(i)
  time.sleep(2)
  article = ""
  for el in driver.find_elements(By.XPATH, "/html/body/div[1]/div/div/div/div/div[1]"):
    count = 0
    for p in el.find_elements(By.TAG_NAME, "div"):
      count += 1
      article += p.text
      article += "\n"
  texts.append(article)
  counts.append(count)

print(texts)
print(counts)

['Ụfọdụ ihe i kwesịrị ịma gbasara \'Chairman\' Christian Chukwu\nEBE FOTO SI,\nGETTY IMAGES\n12 Eprel 2025\nMgbe ikpeazụ e tinyere ya ozi ọhụrụ 16 Eprel 2025\nMgbe ikpeazụ e tinyere ya ozi ọhụrụ 16 Eprel 2025\nA mụrụ ya n\'ụbọchị 4 Jenụwarị 1951 n\'Uwani dị n\'okpuru ọchịchị Enugu South n\'Enugwu steeti.\nỌ gụrụ akwụkwọ praịmarị n\'ụlọakwụkwọ Christ Church Primary School, Uwani ma gụọ nke sekọndịrị na National Secondary School Nike, Enugwu.\nỌ gbara bọọlụ n\'otu egwu bọọlụ Rangers nke Enugwu n\'afọ 1972 ma mechaa soro na ndị nlekọta otu egwu bọọlụ ahụ n\'afọ 1982.\nN\'afọ 1977, o duuru otu egwu bọọlụ Flying Antelopes gaa mabute iko mba Afrịka n\'asọmpi African Cup Winners Cup anaghị agbazi ugbua.\nChristian Chukwu bụ kaptịn Naịjirịa mbụ mabutere iko n\'asọmpi African Nations Cup oge ha jichara ọkpụ goolu atọ asataghị otu merie otu egwu bọọlụ mba Ọljirịa n\'afọ 1980.\nN\'afọ 1985, ọ rụrụ ọrụ dịka osote onye nchịkọta otu egwu bọọlụ Naịjirịa ndị akarịbeghị afọ iri na isii (U-16) dịka Naịj

In [33]:
df = pd.DataFrame(columns = ["text", "category", "source"])
df["text"] = texts
df["category"] = category
df["source"] = links[0]
df

,text,category,source


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Data

/content/drive/MyDrive/Data


In [ ]:
!zip -r scraped_data.zip scraped_data/

  adding: scraped_data/ (stored 0%)
  adding: scraped_data/capacity-building-for-procurement-and-financial-management-project-staff.txt (deflated 56%)
  adding: scraped_data/communication-officers-and-innovation-grant-managers-training-workshop-on-ideas-project.txt (deflated 76%)
  adding: scraped_data/carousel-portfolio-item-layout.txt (deflated 21%)
  adding: scraped_data/essentials-wordpress-theme.txt (stored 0%)
  adding: scraped_data/ideas-project-lunch-in-edo-state.txt (deflated 59%)
  adding: scraped_data/coming-soon.txt (deflated 21%)
  adding: scraped_data/gombe-state-piu.txt (deflated 34%)
  adding: scraped_data/abia-state-piu.txt (deflated 34%)
  adding: scraped_data/services.txt (deflated 71%)
  adding: scraped_data/contact-us-simple.txt (deflated 33%)
  adding: scraped_data/about-nyesaf.txt (stored 0%)
  adding: scraped_data/latest-events.txt (deflated 23%)
  adding: scraped_data/benue-state-piu.txt (deflated 35%)
  adding: scraped_data/beautiful-and-modern-portfolio.txt (

In [ ]:
from google.colab import files
files.download('scraped_data.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Uniccon Scraper

In [ ]:
def get_links(website):

    # initialise driver options
    options = initialize_driver_options()

    # request for a random and different user agent everytime
    user_agent = random.choice(user_agent_list)

    options.add_argument(f'user-agent={user_agent}')

    #initialise the fortified scraper
    driver = webdriver.Chrome(options=options)

    driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": user_agent})

    #enable stealth mode
    stealth(driver,
    languages=["en-US", "en"],
    vendor="Google Inc.",
    platform="Win32",
    webgl_vendor="Intel Inc.",
    renderer="Intel Iris OpenGL Engine",
    fix_hairline=True,
      )


    driver.get(website)
    time.sleep(2)

    #fetch all links in the webpage
    links_list = driver.find_elements(By.TAG_NAME, "a")
    l = []
    time.sleep(2)

    #loop through each link
    for links in links_list:
        time.sleep(2)
        l.append(links.get_attribute('href'))


    #driver.quit()
    return l

In [ ]:
#move to the folder where the scraped data is on your device
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/182DL1ZoX52TDPWfu53xujONm-OQdyT_7/WiChat/Uniccon


In [ ]:
get_links("https://uniccongroup.com/")

['https://uniccongroup.com/',
 'https://uniccongroup.com/blog',
 'https://uniccongroup.com/about-us',
 'https://uniccongroup.com/',
 'https://uniccongroup.com/blog',
 'https://uniccongroup.com/about-us',
 'https://www.youtube.com/watch?si=xEmazZ52w8zqcWq2&v=Tq-lSFCeBx8&feature=youtu.be',
 'https://www.youtube.com/watch?si=xEmazZ52w8zqcWq2&v=Tq-lSFCeBx8&feature=youtu.be',
 'https://chuksprimeorganics.com/',
 'https://omeife.ai/',
 'https://smartafrilabs.com/',
 'https://babasky.com/',
 'https://twitter.com/uniccongroup',
 'https://www.linkedin.com/company/uniccongroup/mycompany/',
 'https://www.facebook.com/Uniccongroup',
 'https://www.instagram.com/uniccongroup/',
 'https://uniccongroup.com/',
 'https://uniccongroup.com/',
 'https://uniccongroup.com/',
 'https://uniccongroup.com/blog',
 'https://uniccongroup.com/about-us',
 'https://twitter.com/uniccongroup',
 'https://www.linkedin.com/company/uniccongroup/mycompany/',
 'https://www.facebook.com/Uniccongroup',
 'https://www.instagram.c

In [ ]:
def scrape(link):
  # initialise driver options
    options = initialize_driver_options()

    # request for a random and different user agent everytime
    user_agent = random.choice(user_agent_list)

    options.add_argument(f'user-agent={user_agent}')

    #initialise the fortified scraper
    driver = webdriver.Chrome(options=options)

    driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": user_agent})

    #enable stealth mode
    stealth(driver,
    languages=["en-US", "en"],
    vendor="Google Inc.",
    platform="Win32",
    webgl_vendor="Intel Inc.",
    renderer="Intel Iris OpenGL Engine",
    fix_hairline=True,
      )


    content = " "
    driver.get(link)
    titles = driver.find_elements(By.TAG_NAME, "h1")
    for t in titles:
        content += t.text + "\n"

    h2 = driver.find_elements(By.TAG_NAME, "h2")
    for t in h2:
        content += t.text + "\n"

    h3 = driver.find_elements(By.TAG_NAME, "h3")
    for t in h3:
        content += t.text + "\n"

    h4 = driver.find_elements(By.TAG_NAME, "h4")
    for t in h4:
        content += t.text + "\n"

    words = driver.find_elements(By.TAG_NAME, "p")
    for t in words:
        content += t.text + "\n"

    save_page_content(link, content)

In [ ]:
# scraping the home page
scrape("https://uniccongroup.com")

In [ ]:
# scraping the about us page
scrape("https://uniccongroup.com/about-us")

In [ ]:
scrape("https://chuksprimeorganics.com")

In [ ]:
scrape("https://omeife.ai/")

In [ ]:
scrape("https://smartafrilabs.com")

In [ ]:
scrape("https://babasky.com/")

We focus on utilizing unmanned aerial vehicle technlogy to provide innovative solutions. Our goal is to transform industries, empower businesses and drive progress through aerial intelligence in Nigeria.

















In [ ]:
scrape("https://smartmedicares.com/")

In [ ]:
scrape("https://omeife.ai/about-us/")

In [ ]:
scrape("https://omeife.ai/hackathon/")

In [ ]:
scrape("https://omeife.ai/features/")